In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## Script to CSV Process

1. Load script `.txt` file.  Use [PDF Extractor](https://pdfextractor.com/) to convert any `.pdf`s to `.txt`s.
2. Remove garbage rows
3. Inspect rows removed
4. Format lines
5. Create `uppercase_unique_form.csv`
6. Manually fill out `uppercase_unique_form.csv` columns `is_character_name` and `is_dialogue` in Excel or other `.csv` editor.
7. Load `uppercase_unique.csv` and filter out rows
8. Inspect rows removed
9. Merge lines into best guesses for columns "character" and "line"
10. Manually clean the `line` column by removing text that is not dialogue.  There will be a lot, so nearly every cell will have to be cleaned!


See the script at the end of this notebook for an example of this process.

## Remove Garbage Rows

In [2]:
def remove_regex_rows(df,string):
    rows_to_remove = df["line"].str.contains(string,case=True,regex=True)
    print("Removed",rows_to_remove.sum(),"rows that match regex \"" + string + "\"")
    
    return df[~rows_to_remove], df[rows_to_remove]

def remove_whitespace_rows(df):
    whitespace_rows = df["line"].str.isspace()
    print(whitespace_rows.sum(),"whitespace rows removed")
    return df[~whitespace_rows], df[whitespace_rows]

def remove_movie_text_rows(df):
    
    df, time_cut_rows = remove_regex_rows(df,"TIME CUT:")
    df, intercut_rows = remove_regex_rows(df,"INTERCUT:")
    df, title_rows = remove_regex_rows(df,"TITLE:")
    df, marvel_rows = remove_regex_rows(df,"MARVEL FLIP")
    df, cut_to_rows = remove_regex_rows(df,"CUT TO")
    df, slam_to_rows = remove_regex_rows(df,"SLAM TO")
    df, dissolve_to_rows = remove_regex_rows(df,"DISSOLVE TO")
    df, the_end_rows = remove_regex_rows(df,"^THE END")
    df, screenplay_rows = remove_regex_rows(df,"BEST ADAPTED SCREENPLAY")
    df, fade_to_black_rows = remove_regex_rows(df,"FADE TO BLACK")
    df, cue_music_rows = remove_regex_rows(df,"CUE MUSIC")
    df, int_rows = remove_regex_rows(df, "^INT\.")
    df, ext_rows = remove_regex_rows(df, "^EXT\.")
    
    removed_rows = pd.concat([time_cut_rows, intercut_rows, title_rows, marvel_rows, 
                              cut_to_rows, slam_to_rows, dissolve_to_rows,
                             the_end_rows, screenplay_rows, fade_to_black_rows,
                             cue_music_rows, int_rows, ext_rows])
    
    return df, removed_rows


def remove_page_number_rows(df):
    page_rows = df["line"].str.isnumeric()
    print(page_rows.sum(),"page number rows removed")
    
    return df[~page_rows], df[page_rows]

def remove_garbage_rows(df):
    df, whitespace_rows = remove_whitespace_rows(df)
    df, page_number_rows = remove_page_number_rows(df)
    df, movie_text_rows = remove_movie_text_rows(df)
    
    garbage_rows = pd.concat([whitespace_rows, page_number_rows, movie_text_rows], 
                            keys = ["whitespace","page_numbers","movie_text"])
    
    print("-----------------------------------------")
    print(garbage_rows.shape[0],"total rows removed\n")
    
    return df, garbage_rows

## Format Character Lines

In [3]:
def remove_regex(df,string):
    print("Removed",df["line"].str.count(string).sum(),"occurences of regex \"" + string + "\"")
    df["line"] = df["line"].str.replace(string,"",case=True,regex=True)

def remove_leading_trailing_whitespace(df):
    df["line"] = df["line"].str.replace("[ \t]+$","")
    df["line"] = df["line"].str.replace("^[ \t]+","")

def format_lines(df):
    remove_leading_trailing_whitespace(df)
    
    remove_regex(df,"\(CONT'D\)")
    remove_regex(df,"\(O\.S\.\)")
    remove_regex(df,"\(O\.S\)")
    remove_regex(df,"\(ON SCREEN\)")
    
    remove_regex(df,"\(OVER COM\)")
    remove_regex(df,"\(ON COM\)")
    remove_regex(df,"\(INTO COM\)")
    remove_regex(df,"\(OVER SPEAKER\)")
    remove_regex(df,"\(INTO PHONE\)")
    remove_regex(df,"\(OVER PHONE\)")
    remove_regex(df,"\(HOLO\)")
    
    print("")
    
    remove_leading_trailing_whitespace(df)

## Remove Non Dialogue Uppercase Rows

In [4]:
def uppercase_rows(df):
    upper_lines = df["line"].str.isupper()
    
    return df[upper_lines]

def create_uppercase_unique_csv(df,script_name):
    '''
    Creates a csv file that is used for inspection of unique
    lines that are all uppercase.  These lines are manually labeled
    to be character names or dialogue.  The resulting table will be
    used to delete rows that are neither character names nor dialogue.
    '''
    
    upper_rows = uppercase_rows(df)
    unique_upper_rows = pd.DataFrame(upper_rows.line.unique())

    unique_upper_rows["words"] = unique_upper_rows[0].str.split(" ").str.len()
    unique_upper_rows["is_character_name"] = 0
    unique_upper_rows["is_dialogue"] = 0
    sorted_unique_upper_rows = unique_upper_rows.sort_values(by=["words"])
    sorted_unique_upper_rows.to_csv("./uppercase forms/" + script_name + "_uppercase_unique_form.csv",encoding='utf-8')
    
def remove_non_dialogue_upper_rows(df,upper_rows):
    '''
    Removes lines that are in all caps, and that are not dialogue or a character's name.
    '''
    unique_character_names = upper_rows[upper_rows["is_character_name"]]["line"]
    unique_dialogues = upper_rows[upper_rows["is_dialogue"]]["line"]
    
    all_character_name_lines = df[df["line"].isin(unique_character_names)]
    all_dialogue_lines = df[df["line"].isin(unique_dialogues)]
    
    print(df["line"].str.isupper().sum(),"total uppercase lines")
    print(all_character_name_lines.shape[0],"uppercase character name lines")
    print(all_dialogue_lines.shape[0],"uppercase dialogue lines")
    
    unique_lines_to_remove = upper_rows[~(upper_rows["is_character_name"]|upper_rows["is_dialogue"])]["line"]
    removed_lines = df["line"].isin(unique_lines_to_remove)
    print(removed_lines.sum(),"uppercase lines removed")
    
    return df[~removed_lines], df[removed_lines] 

## Merge Lines

In [5]:
def is_character_name(string,character_names):
    return string in character_names["line"].tolist()
    

def merge_lines(df,character_names):
    '''
    Starts at a character name and merges all lines until the next character's name.
    The result should be the best guess at columns "character" and "line".
    This will merge in some screenplay text that is not dialogue, so the result will 
    have to be manually cleaned as the final step.
    '''
    merged_df = pd.DataFrame(columns=["character","line"])
    
    for i in range(0,df.shape[0]):
        
        row = df.iloc[i]
        
        if is_character_name(row["line"],character_names):
            character = row["line"]
            line = ""
            
            j = i + 1
            while(j < len(df) and (not is_character_name(df.iloc[j]["line"],character_names))):
                line += df.iloc[j]["line"] + " "
                j += 1
            
            new_row = {"character": character,"line": line}
            
            merged_df = merged_df.append(new_row,ignore_index=True)
    
    return merged_df

## Avengers Endgame

In [6]:
avengers_endgame = pd.read_csv("./script txts/avengers-endgame-script-slug.txt",sep="\n",header=None,names=["line"])

avengers_endgame, garbage_rows = remove_garbage_rows(avengers_endgame)
format_lines(avengers_endgame)

create_uppercase_unique_csv(avengers_endgame,"avengers_endgame")
filled_out_upper_rows = pd.read_csv("./uppercase results/avengers_endgame_uppercase_unique.csv",
                                    names=["","line","words","is_character_name","is_dialogue"],
                                    dtype={"line":str, "words":int, "is_character_name":bool, "is_dialogue":bool},
                                    index_col=0)

avengers_endgame, upper_removed_rows = remove_non_dialogue_upper_rows(avengers_endgame, filled_out_upper_rows)
removed_rows = pd.concat([garbage_rows,pd.concat([upper_removed_rows],keys=[""])],keys=["garbage","uppercase"])


unique_character_names = pd.DataFrame(filled_out_upper_rows[filled_out_upper_rows["is_character_name"]]["line"])

avengers_endgame = merge_lines(avengers_endgame, unique_character_names)
avengers_endgame.reindex(copy=False)

avengers_endgame.to_csv("./uncleaned/avengers_endgame_uncleaned.csv", index=False)

avengers_endgame = pd.read_csv("./cleaned/avengers_endgame.csv")

avengers_endgame.head()

149 whitespace rows removed
290 page number rows removed
Removed 11 rows that match regex "TIME CUT:"
Removed 2 rows that match regex "INTERCUT:"
Removed 8 rows that match regex "TITLE:"
Removed 1 rows that match regex "MARVEL FLIP"
Removed 1 rows that match regex "CUT TO"
Removed 1 rows that match regex "SLAM TO"
Removed 4 rows that match regex "DISSOLVE TO"
Removed 1 rows that match regex "^THE END"
Removed 1 rows that match regex "BEST ADAPTED SCREENPLAY"
Removed 2 rows that match regex "FADE TO BLACK"
Removed 2 rows that match regex "CUE MUSIC"
Removed 95 rows that match regex "^INT\."
Removed 75 rows that match regex "^EXT\."
-----------------------------------------
643 total rows removed

Removed 121 occurences of regex "\(CONT'D\)"
Removed 61 occurences of regex "\(O\.S\.\)"
Removed 1 occurences of regex "\(O\.S\)"
Removed 1 occurences of regex "\(ON SCREEN\)"
Removed 11 occurences of regex "\(OVER COM\)"
Removed 1 occurences of regex "\(ON COM\)"
Removed 28 occurences of regex

,character,line
0,CLINT BARTON,"Okay, you see where you’re going? Let’s work on how to get there."
1,CLINT BARTON,"Okay, good...tip down...bow arm out...three fingers-"
2,LILA BARTON,Why three?
3,CLINT BARTON,‘Cause two’s not enough and four’s too much-
4,LAURA BARTON,"You guys want mustard or mayo, or both?"
